In [3]:
"""
1.	Architecture (layer_dims = [2,16,16,8,8,1])
	•	Input layer: size 2
	•	Hidden 1: size 16 (ReLU + BatchNorm)
	•	Hidden 2: size 16 (ReLU + BatchNorm)
	•	Hidden 3: size 8  (ReLU + BatchNorm)
	•	Hidden 4: size 8  (ReLU + BatchNorm)
	•	Output: size 1 (Sigmoid)
2.	Batch Normalization
	•	Each hidden layer ℓ (ℓ=1…4) has a BatchNorm instance.
3.	Initialization
	•	Hidden layers use He initialization (np.random.randn(...) * sqrt(2/n_in)).
	•	Output layer uses small random normal (* 0.01).
4.	Optimizer: Adam
	•	We keep m and v for each parameter (Wℓ, bℓ, γℓ, βℓ) and perform bias‐corrected updates each epoch.
	•	Learning rate lr=0.001.
5.	Training Loop
	•	For each epoch:
	•	Forward pass (A_L, cache = model.forward(X_train))
	•	Compute loss & grads (loss, grads = model.compute_loss_and_grads(...))
	•	Update parameters with Adam (model.update_params_adam(grads))
	•	Every 20 epochs (and for epoch 1), we print training and validation accuracy.

    """

'\n1.\tArchitecture (layer_dims = [2,16,16,8,8,1])\n\t•\tInput layer: size 2\n\t•\tHidden 1: size 16 (ReLU + BatchNorm)\n\t•\tHidden 2: size 16 (ReLU + BatchNorm)\n\t•\tHidden 3: size 8  (ReLU + BatchNorm)\n\t•\tHidden 4: size 8  (ReLU + BatchNorm)\n\t•\tOutput: size 1 (Sigmoid)\n2.\tBatch Normalization\n\t•\tEach hidden layer ℓ (ℓ=1…4) has a BatchNorm instance.\n3.\tInitialization\n\t•\tHidden layers use He initialization (np.random.randn(...) * sqrt(2/n_in)).\n\t•\tOutput layer uses small random normal (* 0.01).\n4.\tOptimizer: Adam\n\t•\tWe keep m and v for each parameter (Wℓ, bℓ, γℓ, βℓ) and perform bias‐corrected updates each epoch.\n\t•\tLearning rate lr=0.001.\n5.\tTraining Loop\n\t•\tFor each epoch:\n\t•\tForward pass (A_L, cache = model.forward(X_train))\n\t•\tCompute loss & grads (loss, grads = model.compute_loss_and_grads(...))\n\t•\tUpdate parameters with Adam (model.update_params_adam(grads))\n\t•\tEvery 20 epochs (and for epoch 1), we print training and validation accurac

In [4]:
import numpy as np

# ----------------------------------------
# 1) Synthetic Binary Dataset (reuse)
# ----------------------------------------
def generate_binary_data(n_per_class=500, seed=42):
    """
    Two Gaussian blobs in 2D for class 0 and class 1.
    Returns:
      X: shape (2*n_per_class, 2)
      Y: shape (2*n_per_class, 1) with values {0,1}
    """
    np.random.seed(seed)
    N = n_per_class
    cov = [[0.5, 0], [0, 0.5]]

    # Class 0 centered at (-1, -1)
    x0 = np.random.multivariate_normal(mean=[-1, -1], cov=cov, size=N)
    y0 = np.zeros((N, 1))

    # Class 1 centered at (+1, +1)
    x1 = np.random.multivariate_normal(mean=[1, 1], cov=cov, size=N)
    y1 = np.ones((N, 1))

    X = np.vstack([x0, x1])  # (2N, 2)
    Y = np.vstack([y0, y1])  # (2N, 1)

    perm = np.random.permutation(2 * N)
    return X[perm], Y[perm]


# ----------------------------------------
# 2) Batch Normalization Layer
# ----------------------------------------
class BatchNorm:
    def __init__(self, num_features, eps=1e-5, momentum=0.9):
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum

        # Learnable scale (γ) and shift (β)
        self.gamma = np.ones((1, num_features))
        self.beta = np.zeros((1, num_features))

        # Running (EMA) statistics for inference
        self.running_mean = np.zeros((1, num_features))
        self.running_var = np.ones((1, num_features))

        # Cache for backprop
        self.cache = None

    def forward(self, X, training=True):
        """
        X: shape (batch, num_features)
        Returns: normalized and scaled output, same shape
        """
        if training:
            batch_mean = np.mean(X, axis=0, keepdims=True)       # (1, F)
            batch_var = np.var(X, axis=0, keepdims=True)         # (1, F)
            X_centered = X - batch_mean                          # (batch, F)
            inv_std = 1.0 / np.sqrt(batch_var + self.eps)        # (1, F)
            X_hat = X_centered * inv_std                         # (batch, F)
            out = self.gamma * X_hat + self.beta                 # (batch, F)

            # Update running stats
            self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * batch_mean
            self.running_var  = self.momentum * self.running_var  + (1 - self.momentum) * batch_var

            # Cache for backward
            self.cache = (X_hat, inv_std, X_centered, batch_var, X.shape[0])
            return out
        else:
            X_centered = X - self.running_mean
            X_hat = X_centered / np.sqrt(self.running_var + self.eps)
            return self.gamma * X_hat + self.beta

    def backward(self, dout):
        """
        dout: upstream gradient, shape (batch, num_features)
        Returns: dX (same shape), and sets self.dgamma, self.dbeta
        """
        X_hat, inv_std, X_centered, var, m = self.cache

        # Gradients for gamma, beta
        self.dgamma = np.sum(dout * X_hat, axis=0, keepdims=True)  # (1, F)
        self.dbeta  = np.sum(dout, axis=0, keepdims=True)           # (1, F)

        # Backprop through normalization
        dX_hat = dout * self.gamma                                   # (batch, F)
        dvar = np.sum(dX_hat * X_centered * (-0.5) * inv_std**3, axis=0, keepdims=True)  # (1, F)
        dmean = (np.sum(dX_hat * -inv_std, axis=0, keepdims=True) +
                 dvar * np.mean(-2 * X_centered, axis=0, keepdims=True))               # (1, F)
        dX = (dX_hat * inv_std) + (dvar * 2 * X_centered / m) + (dmean / m)             # (batch, F)
        return dX


# ----------------------------------------
# 3) Deep MLP for Binary Classification (Corrected)
# ----------------------------------------
class DeepMLP:
    def __init__(self, layer_dims, lr=0.001, seed=0):
        """
        layer_dims: list of dims, e.g. [2, 16, 16, 8, 8, 1]
        (Here: 2 inputs → 4 hidden layers → 1 output)
        """
        np.random.seed(seed)
        self.lr = lr
        self.L = len(layer_dims) - 1  # number of trainable layers

        # Parameters: Wℓ, bℓ, and BatchNorm ℓ for hidden layers (ℓ=1...L-1)
        self.params = {}
        self.batchnorms = {}
        for ℓ in range(1, self.L + 1):
            n_in = layer_dims[ℓ - 1]
            n_out = layer_dims[ℓ]

            if ℓ < self.L:
                # Hidden layer → He initialization for ReLU
                self.params[f"W{ℓ}"] = np.random.randn(n_in, n_out) * np.sqrt(2.0 / n_in)
            else:
                # Output layer → small random
                self.params[f"W{ℓ}"] = np.random.randn(n_in, n_out) * 0.01

            self.params[f"b{ℓ}"] = np.zeros((1, n_out))

            # For hidden layers ℓ < L, create BatchNorm
            if ℓ < self.L:
                self.batchnorms[f"BN{ℓ}"] = BatchNorm(num_features=n_out)

        # Adam optimizer state: m, v for Wℓ, bℓ, gammaℓ, betaℓ
        self.adam_m = {}
        self.adam_v = {}
        for ℓ in range(1, self.L + 1):
            self.adam_m[f"W{ℓ}"] = np.zeros_like(self.params[f"W{ℓ}"])
            self.adam_v[f"W{ℓ}"] = np.zeros_like(self.params[f"W{ℓ}"])
            self.adam_m[f"b{ℓ}"] = np.zeros_like(self.params[f"b{ℓ}"])
            self.adam_v[f"b{ℓ}"] = np.zeros_like(self.params[f"b{ℓ}"])
        for ℓ in range(1, self.L):
            self.adam_m[f"gamma{ℓ}"] = np.zeros_like(self.batchnorms[f"BN{ℓ}"].gamma)
            self.adam_v[f"gamma{ℓ}"] = np.zeros_like(self.batchnorms[f"BN{ℓ}"].gamma)
            self.adam_m[f"beta{ℓ}"]  = np.zeros_like(self.batchnorms[f"BN{ℓ}"].beta)
            self.adam_v[f"beta{ℓ}"]  = np.zeros_like(self.batchnorms[f"BN{ℓ}"].beta)

        self.adam_t = 0
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.eps = 1e-8

    def relu(self, Z):
        return np.maximum(0, Z)

    def relu_grad(self, Z):
        return (Z > 0).astype(float)

    def sigmoid(self, Z):
        return 1.0 / (1.0 + np.exp(-Z))

    def forward(self, X, training=True):
        """
        X: (batch, input_dim)
        Returns: A_L (batch,1) and caches intermediate values.
        """
        cache = {}
        A_prev = X
        cache["A0"] = X

        # Hidden layers ℓ = 1 ... L-1
        for ℓ in range(1, self.L):
            Wℓ = self.params[f"W{ℓ}"]
            bℓ = self.params[f"b{ℓ}"]

            # 1) Linear
            Zℓ = A_prev.dot(Wℓ) + bℓ                   # (batch, hidden_dim)

            # 2) BatchNorm
            BNℓ = self.batchnorms[f"BN{ℓ}"]
            Zℓ_norm = BNℓ.forward(Zℓ, training=training)  # (batch, hidden_dim)

            # 3) ReLU
            Aℓ = self.relu(Zℓ_norm)                      # (batch, hidden_dim)

            cache[f"Z{ℓ}"]      = Zℓ
            cache[f"Znorm{ℓ}"]  = Zℓ_norm
            cache[f"A{ℓ}"]      = Aℓ
            A_prev = Aℓ

        # Output layer ℓ = L
        Wℓ = self.params[f"W{self.L}"]
        bℓ = self.params[f"b{self.L}"]
        Zℓ = A_prev.dot(Wℓ) + bℓ                        # (batch, 1)
        Aℓ = self.sigmoid(Zℓ)                          # (batch, 1)

        cache[f"Z{self.L}"] = Zℓ
        cache[f"A{self.L}"] = Aℓ

        return Aℓ, cache

    def compute_loss_and_grads(self, A_L, Y, cache):
        """
        Compute binary cross‐entropy loss and backprop gradients.
        A_L: (batch,1), Y: (batch,1)
        cache: intermediate values
        Returns: loss, grads dict (dWℓ, dbℓ, dgammaℓ, dbetaℓ)
        """
        m = Y.shape[0]
        # Compute loss
        A_L_clipped = np.clip(A_L, 1e-8, 1 - 1e-8)
        loss = -np.sum(Y * np.log(A_L_clipped) + (1 - Y) * np.log(1 - A_L_clipped)) / m

        grads = {}
        # -------------------
        # Output layer ℓ = L
        # -------------------
        ZL = cache[f"Z{self.L}"]             # (batch,1)
        A_prev = cache[f"A{self.L - 1}"]     # (batch, hidden_dim of L-1)
        dZL = A_L - Y                        # (batch,1)
        grads[f"dW{self.L}"] = A_prev.T.dot(dZL) / m  # (hidden_dim, 1)
        grads[f"db{self.L}"] = np.sum(dZL, axis=0, keepdims=True) / m   # (1,1)
        dA_prev = dZL.dot(self.params[f"W{self.L}"].T)       # (batch, hidden_dim)

        # -----------------------------------
        # Hidden layers ℓ = L-1 ... 1 (reverse)
        # -----------------------------------
        for ℓ in reversed(range(1, self.L)):
            Zℓ      = cache[f"Z{ℓ}"]                # (batch, hidden_dim)
            Zℓ_norm = cache[f"Znorm{ℓ}"]            # (batch, hidden_dim)
            A_prev  = cache[f"A{ℓ - 1}"]             # (batch, size of layer ℓ-1)

            # 1) Backprop through ReLU
            dZ_norm = dA_prev * self.relu_grad(Zℓ_norm)  # (batch, hidden_dim)

            # 2) Backprop through BatchNorm
            BNℓ = self.batchnorms[f"BN{ℓ}"]
            dZ  = BNℓ.backward(dZ_norm)                  # (batch, hidden_dim)
            grads[f"dgamma{ℓ}"] = BNℓ.dgamma             # (1, hidden_dim)
            grads[f"dbeta{ℓ}"]  = BNℓ.dbeta              # (1, hidden_dim)

            # 3) Backprop through linear
            grads[f"dW{ℓ}"] = A_prev.T.dot(dZ) / m       # (size of layer ℓ-1, hidden_dim)
            grads[f"db{ℓ}"] = np.sum(dZ, axis=0, keepdims=True) / m     # (1, hidden_dim)
            dA_prev = dZ.dot(self.params[f"W{ℓ}"].T)     # (batch, size of layer ℓ-1)

        return loss, grads

    def update_params_adam(self, grads):
        """
        Update all parameters with Adam optimizer.
        grads: dictionary containing gradients for dWℓ, dbℓ, dgammaℓ, dbetaℓ.
        """
        self.adam_t += 1
        lr_t = self.lr * np.sqrt(1 - self.beta2**self.adam_t) / (1 - self.beta1**self.adam_t)

        # Update weights and biases
        for ℓ in range(1, self.L + 1):
            for param in [f"W{ℓ}", f"b{ℓ}"]:
                grad = grads[f"d{param}"]
                # Update biased first moment
                self.adam_m[param] = self.beta1 * self.adam_m[param] + (1 - self.beta1) * grad
                # Update biased second moment
                self.adam_v[param] = self.beta2 * self.adam_v[param] + (1 - self.beta2) * (grad ** 2)
                # Compute bias‐corrected estimates
                m_hat = self.adam_m[param] / (1 - self.beta1**self.adam_t)
                v_hat = self.adam_v[param] / (1 - self.beta2**self.adam_t)
                # Update parameter
                self.params[param] -= lr_t * m_hat / (np.sqrt(v_hat) + self.eps)

        # Update batchnorm γ and β for ℓ = 1...L-1
        for ℓ in range(1, self.L):
            for param in [f"gamma{ℓ}", f"beta{ℓ}"]:
                grad_key = "d" + param       # e.g. "dgamma1" or "dbeta1"
                grad = grads[grad_key]
                self.adam_m[param] = self.beta1 * self.adam_m[param] + (1 - self.beta1) * grad
                self.adam_v[param] = self.beta2 * self.adam_v[param] + (1 - self.beta2) * (grad ** 2)
                m_hat = self.adam_m[param] / (1 - self.beta1**self.adam_t)
                v_hat = self.adam_v[param] / (1 - self.beta2**self.adam_t)
                if param.startswith("gamma"):
                    ℓ_idx = int(param.replace("gamma", ""))
                    self.batchnorms[f"BN{ℓ_idx}"].gamma -= lr_t * m_hat / (np.sqrt(v_hat) + self.eps)
                else:  # "beta"
                    ℓ_idx = int(param.replace("beta", ""))
                    self.batchnorms[f"BN{ℓ_idx}"].beta -= lr_t * m_hat / (np.sqrt(v_hat) + self.eps)

    def predict(self, X):
        A_L, _ = self.forward(X, training=False)
        return (A_L > 0.5).astype(int)


# ----------------------------------------
# 4) Training Loop for Deep MLP Binary
# ----------------------------------------
if __name__ == "__main__":
    # Generate data
    X, Y = generate_binary_data(n_per_class=500, seed=42)
    # Split 80% train, 20% val
    split = int(0.8 * X.shape[0])
    X_train, Y_train = X[:split], Y[:split]
    X_val,   Y_val   = X[split:], Y[split:]

    # Define network dims: 2→16→16→8→8→1 (4 hidden layers)
    layer_dims = [2, 16, 16, 8, 8, 1]
    model = DeepMLP(layer_dims, lr=0.001, seed=1)

    epochs = 200
    for epoch in range(1, epochs + 1):
        # Forward + compute loss + backward
        A_L, cache = model.forward(X_train, training=True)
        loss, grads = model.compute_loss_and_grads(A_L, Y_train, cache)
        model.update_params_adam(grads)

        if epoch % 20 == 0 or epoch == 1:
            train_preds = model.predict(X_train)
            val_preds   = model.predict(X_val)
            train_acc = np.mean(train_preds == Y_train)
            val_acc   = np.mean(val_preds   == Y_val)
            print(f"Epoch {epoch:3d} | Loss: {loss:.4f} | "
                  f"Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    # Final evaluation
    train_acc = np.mean(model.predict(X_train) == Y_train)
    val_acc   = np.mean(model.predict(X_val)   == Y_val)
    print(f"\nFinal Train Acc: {train_acc:.4f} | Final Val Acc: {val_acc:.4f}")

Epoch   1 | Loss: 0.7014 | Train Acc: 0.4788 | Val Acc: 0.5350
Epoch  20 | Loss: 0.6974 | Train Acc: 0.2725 | Val Acc: 0.2150
Epoch  40 | Loss: 0.6942 | Train Acc: 0.4037 | Val Acc: 0.3450
Epoch  60 | Loss: 0.6916 | Train Acc: 0.5100 | Val Acc: 0.4600
Epoch  80 | Loss: 0.6884 | Train Acc: 0.5100 | Val Acc: 0.4600
Epoch 100 | Loss: 0.6826 | Train Acc: 0.5975 | Val Acc: 0.5600
Epoch 120 | Loss: 0.6728 | Train Acc: 0.7550 | Val Acc: 0.7550
Epoch 140 | Loss: 0.6588 | Train Acc: 0.8475 | Val Acc: 0.8200
Epoch 160 | Loss: 0.6410 | Train Acc: 0.8925 | Val Acc: 0.8550
Epoch 180 | Loss: 0.6197 | Train Acc: 0.9325 | Val Acc: 0.8950
Epoch 200 | Loss: 0.5958 | Train Acc: 0.9587 | Val Acc: 0.9450

Final Train Acc: 0.9587 | Final Val Acc: 0.9450
